In [ ]:
# ==============================================================================
# 1. SETUP & CONFIGURATION
# ==============================================================================
# Cài đặt thư viện (Chỉ chạy lần đầu)
# !pip install pytorch-forecasting pytorch-lightning polars --quiet

import pandas as pd
import polars as pl
import numpy as np
import os
from datetime import timedelta

# Pytorch
import torch
import torch.nn as nn
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint

# Pytorch Forecasting
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss

# Thiết lập Random Seed để tái tạo kết quả
SEED = 42
seed_everything(SEED, workers=True)

# Cấu hình đường dẫn
DATA_PATH = "/kaggle/input/financial-data-ohlcv-global/dataset_final_kaggle.parquet"
CHECKPOINT_PATH = "/kaggle/working/tft_v1"

# Tham số mô hình
MAX_ENCODER_LENGTH = 30 # Nhìn lại 30 ngày quá khứ
MAX_PREDICTION_LENGTH = 5 # Dự báo 5 ngày tương lai
BATCH_SIZE = 128
MAX_EPOCHS = 50 
LEARNING_RATE = 1e-3

In [ ]:
# ==============================================================================
# 2. LOAD DATA & PREPROCESSING
# ==============================================================================

print("1. Đang tải và tiền xử lý dữ liệu...")
df_pl = pl.read_parquet(DATA_PATH)
df = df_pl.to_pandas()

# Chuyển đổi và làm sạch cuối cùng (Đảm bảo đúng type cho Pytorch)
df = df.sort_values(by=['symbol', 'ts']).reset_index(drop=True)
df['symbol_id'] = df['symbol'].astype('category').cat.codes
df['asset_type'] = df['asset_type'].astype('category').cat.codes

# Loại bỏ các mã ít dữ liệu (Dưới 1 năm, để tăng chất lượng)
symbols_to_keep = df.groupby('symbol').size().nlargest(3000).index # Giữ 3000 mã tốt nhất
df = df[df['symbol'].isin(symbols_to_keep)].copy()

print(f"   -> Kích thước tập dữ liệu cuối cùng: {df.shape[0]:,} dòng ({len(symbols_to_keep)} mã)")

# Chia tập Train/Validation theo THỜI GIAN
# Giữ 5 ngày dự báo + 10 ngày cho Encoder (tổng 15 ngày) để đánh giá.
training_cutoff = df["time_idx"].max() - MAX_PREDICTION_LENGTH * 3 
print(f"   -> Cắt dữ liệu tại time_idx: {training_cutoff}")

In [ ]:
# ==============================================================================
# 3. ĐỊNH NGHĨA TIMESERIESDATASET
# ==============================================================================

print("2. Đang cấu hình TimeSeriesDataSet...")

training = TimeSeriesDataSet(
    df[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="log_return", # Mục tiêu: Dự báo lợi nhuận
    group_ids=["symbol_id"], # Nhóm: Multi-asset learning
    
    max_encoder_length=MAX_ENCODER_LENGTH,
    max_prediction_length=MAX_PREDICTION_LENGTH,
    
    # 3a. STATIC (Cố định):
    static_categoricals=["symbol_id", "asset_type"],
    
    # 3b. KNOWN REAL (Biết trước tương lai):
    time_varying_known_reals=[
        "time_idx", "day_sin", "day_cos", "month_sin", "month_cos"
    ],
    
    # 3c. UNKNOWN REAL (Quan sát quá khứ):
    time_varying_unknown_reals=[
        "log_return", "vol_relative", "bb_width", "roc_10", "macd_proxy", # Indicators
        "ctx_sp500_ret", "ctx_gold_ret", "ctx_oil_ret", "ctx_forex_ret",  # Global Context
        "ctx_sp500_vol", "ctx_gold_vol", "ctx_oil_vol", "ctx_forex_vol",  # Global Volume
    ],
    
    # Chuẩn hóa riêng cho từng mã
    target_normalizer=GroupNormalizer(groups=["symbol_id"], center=False, scale_by_group=True),
    add_target_scales=True,
    add_encoder_length=True,
)

# Tạo Validation Dataset
validation = TimeSeriesDataSet.from_dataset(training, df, min_prediction_idx=training_cutoff + 1, overwrite_existing_dataset_fields=True)

# Tạo DataLoaders (Tăng tốc đọc dữ liệu)
train_dataloader = training.to_dataloader(train=True, batch_size=BATCH_SIZE, num_workers=4)
val_dataloader = validation.to_dataloader(train=False, batch_size=BATCH_SIZE, num_workers=4)

In [ ]:
# ==============================================================================
# 4. KHỞI TẠO VÀ HUẤN LUYỆN MODEL TFT
# ==============================================================================

print("3. Đang cấu hình TFT Model và Callbacks...")

# Callbacks:
# 1. Early Stopping: Dừng nếu val_loss không cải thiện trong 7 epochs
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=7, verbose=False, mode="min")
# 2. Model Checkpoint: Lưu model tốt nhất (val_loss thấp nhất)
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath=CHECKPOINT_PATH,
    filename="best_tft_v1-{epoch:02d}-{val_loss:.4f}",
    save_top_k=1,
    mode="min",
    verbose=True
)

# Khởi tạo TFT Model
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=LEARNING_RATE,
    hidden_size=64,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=64,
    output_size=7, # 7 Quantiles cho QuantileLoss
    loss=QuantileLoss(),
    log_interval=10,
    optimizer="adam",
    reduce_on_plateau_patience=4,
)

# Khởi tạo Trainer
trainer = Trainer(
    max_epochs=MAX_EPOCHS,
    accelerator="gpu",
    devices=1,
    gradient_clip_val=0.1,
    callbacks=[early_stop_callback, checkpoint_callback, LearningRateMonitor()],
)

print("\n🚀 BẮT ĐẦU TRAINING BASE TFT (Version 1.0)...")
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

# Lưu Model cuối cùng (Nếu không muốn dùng checkpoint)
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
print(f"✅ Training hoàn thành! Model tốt nhất được lưu tại: {best_model_path}")

In [ ]:
# ==============================================================================
# 5. DỰ BÁO & VISUALIZATION (Sau khi Training)
# ==============================================================================

print("\n4. Đang kiểm tra dự báo trên tập Validation...")

# Chọn một vài mã ngẫu nhiên để vẽ biểu đồ
target_symbols = validation.data.symbol.unique()
# Chọn 3 mã bất kỳ
symbols_to_plot = np.random.choice(target_symbols, 3, replace=False)

for sym in symbols_to_plot:
    # Lọc dữ liệu của mã đó
    encoder_data = df[lambda x: (x.symbol == sym) & (x.time_idx <= training_cutoff)]
    last_encoder_data = encoder_data[encoder_data.time_idx == encoder_data.time_idx.max()]
    
    # Lấy dữ liệu validation cho mã đó
    validation_data = df[lambda x: (x.symbol == sym) & (x.time_idx > training_cutoff)]
    
    # Dự báo
    raw_predictions = best_tft.predict(last_encoder_data, mode="raw", return_x=True)
    
    # Plotting (sử dụng thư viện nội bộ của Pytorch Forecasting)
    fig = best_tft.plot_prediction(
        raw_predictions, 
        x=raw_predictions.x, 
        add_loss_to_title=True, 
        show_future_observed_values=True,
        ax_kwargs={"title": f"Dự báo 5 ngày cho Mã: {sym}"}
    )
    fig.show() # Trên Kaggle, bạn sẽ thấy biểu đồ hiển thị ngay dưới cell này